In [ ]:
import numpy as np
from numpy.linalg import matrix_power
from numpy import dot, einsum, array, arange, stack, ones, array_equal
from itertools import product as iter_product

M = 4
N = M**2

def inv(x):
    return np.linalg.inv(x).astype(np.int)

def mod(g):
    return np.array([g.T[0], g.T[1], g.T[2] % M]).T

def plot(g):
    coords = stack(np.unravel_index(arange(N), (M, M))+(ones(N, dtype=np.int),), 1)
    transformed = np.einsum('ab,nb->na', g, coords)
    res = np.zeros((M, M), dtype=np.int)-1
    for n, (i, j, _) in enumerate(transformed):
        res[i % M, j % M] = n
    return res

def product(A, B):
    return mod(stack((dot(a, b) for a, b in iter_product(A, B)), 0))

def generate(a, n):
    return mod(stack((matrix_power(a, i) for i in range(n)), 0))

def order(G):
    return len(set([np.array_str(g) for g in G]))

def same(A, B):
    A_str = sorted([np.array_str(a) for a in mod(A)])
    B_str = sorted([np.array_str(b) for b in mod(B)])
    return A_str == B_str

def neighbours(grid):
    grid_flat = grid.flatten()
    coords = np.stack(np.unravel_index(arange(N), (M, M)), 1)
    offset = np.array([[0, 1], [0, -1], [1, 0], [-1, 0]])
    neighbour_coords = coords[:, np.newaxis, :] + offset
    neighbours = grid_flat[np.ravel_multi_index(np.transpose(neighbour_coords, (2, 0, 1)), (M, M), 'wrap')]
    return {grid_flat[i]: sorted(neighbours[i]) for i in range(N)}
    
r0 = array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
m0 = array([[1, 0, 0], [0, -1, 0], [0, 0, 1]])
D4 = product(generate(r0, 4), generate(m0, 2))

i0 = array([[1, 0, 1], [0, 1, 0], [0, 0, 1]])
j0 = array([[1, 0, 0], [0, 1, 1], [0, 0, 1]])
T = product(generate(i0, M), generate(j0, M))

G = product(D4, T)

# Group axioms

# 1. Closure
for a, b in iter_product(G, G):
    assert mod(np.dot(a, b)) in G
    
# 2. Associativity
for a, b, c in iter_product(G, G, G):
    assert array_equal(mod(np.dot(a, mod(np.dot(b, c)))), mod(np.dot(mod(np.dot(a, b)), c)))

# 3. Identity is identity matrix
    
# 4. Inverse
for g in G:
    assert mod(inv(g)) in G
    assert array_equal(mod(mod(inv(g)).dot(g)), np.eye(3))
    assert array_equal(mod(g.dot(mod(inv(g)))), np.eye(3))
    
# Product sane
assert same(product(T, D4), product(D4, T))
assert order(T) * order(D4) == order(product(T, D4)) == 8 * M**2

# Product well defined: gt' = gt if t'=t
for g, t in iter_product(G, T):
    for p, q in iter_product([-1, 0, 1], [-1, 0, 1]):
        offset = np.dot(matrix_power(i0, M*p), matrix_power(j0, M*q))
        t_prime = np.dot(t, offset)
    assert array_equal(mod(np.dot(g, t)), mod(np.dot(g, t_prime)))

# Normal subgroup
for t, g in iter_product(T, G):
    assert mod(np.dot(mod(np.dot(g, t)), inv(g))) in T
    
# Symmetry on periodic lattice (induces bijection)
for g in G:
    assert (plot(g) == -1).sum() == 0
    
# Preserves neighbours
identity_neighbours = neighbours(plot(T[0]))
for g in G:
    assert neighbours(plot(g)) == identity_neighbours
    
print("Assertions succesful")

In [267]:
grid = plot(mod(np.dot(D4[7], T[5])))
grid

array([[65, 55, 45, 35, 25, 15,  5, 95, 85, 75],
       [64, 54, 44, 34, 24, 14,  4, 94, 84, 74],
       [63, 53, 43, 33, 23, 13,  3, 93, 83, 73],
       [62, 52, 42, 32, 22, 12,  2, 92, 82, 72],
       [61, 51, 41, 31, 21, 11,  1, 91, 81, 71],
       [60, 50, 40, 30, 20, 10,  0, 90, 80, 70],
       [69, 59, 49, 39, 29, 19,  9, 99, 89, 79],
       [68, 58, 48, 38, 28, 18,  8, 98, 88, 78],
       [67, 57, 47, 37, 27, 17,  7, 97, 87, 77],
       [66, 56, 46, 36, 26, 16,  6, 96, 86, 76]])

In [270]:
neighbours(grid)[12]

[2, 11, 13, 22]